In [2]:
!pip install openai==0.28

  Obtaining dependency information for openai==0.28 from https://files.pythonhosted.org/packages/ae/59/911d6e5f1d7514d79c527067643376cddcf4cb8d1728e599b3b03ab51c69/openai-0.28.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ---------------- ----------------------- 30.7/76.5 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 76.5/76.5 kB 850.1 kB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.0.5 requires langchain-core<0.2,>=0.1.16, but you have langchain-core 0.3.13 which is incompatible.
langchain-openai 0.0.5 requires openai<2.0.0,>=1.10.0, but you have openai 0.28.0 which is incompatible.


In [8]:
# Install necessary libraries
#!pip install openai datasets scikit-learn

import openai
import os
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm  # For progress bar

# Set your OpenAI API key
openai.api_key = "YOUR_OPENAI_API_KEY"


In [18]:
# Load the AG News test dataset
dataset = load_dataset("ag_news", split='test')

# Define label mappings
id2label = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
label2id = {v: k for k, v in id2label.items()}

# For demonstration purposes, we'll use a subset to limit API calls
subset_size = 500  # Adjust this number as needed
test_texts = dataset['text'][:subset_size]
true_labels = dataset['label'][:subset_size]

In [19]:

# Function to get GPT-4 classification
def classify_text_with_gpt4(text):
    prompt = f"""
    Please classify the following news article into one of the following categories: World, Sports, Business, Sci/Tech.

    Article:
    \"\"\"
    {text}
    \"\"\"

    Provide only the category name.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0  # Set temperature to 0 for deterministic output
        )

        # Extract the assistant's reply
        classification = response['choices'][0]['message']['content'].strip()
        return classification
    except Exception as e:
        print(f"Error: {e}")
        return None

In [20]:
# Initialize list to store GPT-4 predictions
gpt4_predictions = []

# Loop through test texts and get GPT-4 predictions
for i, text in enumerate(tqdm(test_texts, desc="Processing samples")):
    predicted_label = classify_text_with_gpt4(text)
    gpt4_predictions.append(predicted_label)

Processing samples:  70%|██████████████████████████████████████████                  | 351/500 [03:19<01:29,  1.66it/s]

Error: The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_7cc6afed08c50250bec41ee9118a3643 in your email.) {
  "error": {
    "message": "The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_7cc6afed08c50250bec41ee9118a3643 in your email.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
} 500 {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_7cc6afed08c50250bec41ee9118a3643 in your email.)', 'type': 'server_error', 'param': None, 'code': None}} {'Date

Processing samples: 100%|████████████████████████████████████████████████████████████| 500/500 [04:45<00:00,  1.75it/s]


In [23]:



# Map GPT-4 predictions to label IDs
predicted_label_ids = []
for pred in gpt4_predictions:
    if pred in label2id:
        predicted_label_ids.append(label2id[pred])
    else:
        predicted_label_ids.append(-1)  # Use -1 for invalid predictions

# Remove samples where GPT-4 did not provide a valid category
valid_indices = [i for i, label_id in enumerate(predicted_label_ids) if label_id != -1]
valid_true_labels = [true_labels[i] for i in valid_indices]
valid_predicted_labels = [predicted_label_ids[i] for i in valid_indices]

# Compute evaluation metrics
precision, recall, f1, _ = precision_recall_fscore_support(
    valid_true_labels, valid_predicted_labels, average='weighted'
)
acc = accuracy_score(valid_true_labels, valid_predicted_labels)

print(f"\nEvaluation Results on {len(valid_true_labels)} samples:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")



Evaluation Results on 499 samples:
Accuracy: 0.8717
Precision: 0.8886
Recall: 0.8717
F1 Score: 0.8723


In [24]:
len(gpt4_predictions),len(valid_indices)

(500, 499)